#### First Implementation of scraping all events of defined Website Scrapers and Visualize them in Qgrid

In [1]:
import datetime as dt
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Button, Dropdown, FloatText, BoundedFloatText, BoundedIntText, Label, ToggleButtons, HTML, Checkbox, AppLayout, Layout, DatePicker
import webbrowser
from src import ScraperCollection
from datetime import datetime, timedelta

In [2]:
checkbox_layout = {'width': '20px'}
label_layout  = {'width': '200px'}
button_layout = {'width': '100px'}
widget_rows = [{
#    'check': Checkbox(value=False, descrption='', indent=False, layout=checkbox_layout),
    'label': Label(scraper.name, layout = label_layout),
    'start_scraper': Button(description='get events', layout = button_layout, disabled = not scraper.ready),
    'open_url': Button(description='open url', layout = button_layout)
} for scraper in ScraperCollection.scrapers]

end_date_widget = DatePicker(
    description = 'End date',
    value = (datetime.now().date() + timedelta(7))
)
HBox_list = [HBox(list(widget_row.values())) for widget_row in widget_rows]
app = VBox([end_date_widget, *HBox_list])

def open_url_func(url):
    def open_url(button):
        webbrowser.open_new_tab(url)
    return open_url

def change_color(button):
    button.button_style = 'success' 

for i, widget_row in enumerate(widget_rows):
    url_button = widget_row['open_url']
    scraper_button = widget_row['start_scraper']
    url_button.on_click(open_url_func(ScraperCollection.scrapers[i].url))
    url_button.on_click(change_color)
    scraper_button.on_click(change_color)

app

In [4]:
# end_datetime = datetime.combine(end_date_widget.value, datetime.min.time())
# print(end_datetime)
# events = ScraperCollection.get_all_events_df(end_datetime)

2023-09-11 00:00:00
JapanischesPalais Scraper started.
SLUB Scraper started.
Lokale Agenda Scraper started.


ValueError: time data 'Fr. 31. Mrz 2023' does not match format '%a. %d. %b %Y'

In [ ]:
# cell_renderer_checkbox = '''function(params) { 
#     var input = document.createElement('input');
#     input.type="checkbox";
#     input.checked=params.value;
#     input.addEventListener('click', function (event) {
#       params.value=!params.value;
#       params.node.data.fieldName = params.value;
#     });
#     return input;
#   }
# '''

# date_picker = '''function DatePicker() {}
# DatePicker.prototype.init = function(params) {
#   this.eInput = document.createElement('input');
#   this.eInput.type = 'date';
#   this.eInput.value = params.value;
#   this.params = params;
#   var that = this;
#   this.eInput.addEventListener('input', function(event) {
#     that.params.setValue(event.target.value);
#   });
# };
# DatePicker.prototype.getGui = function() {
#   return this.eInput;
# };
# '''
# column_defs = [{'headerName':'Start','field':'start_date', 'cellDataType': 'dateString', 'editable': True},
#                {'headerName':'Typ','field':'event_type', 'editable': True},
#                {'headerName':'Titel','field':'title'},
#                {'headerName':'URL','field':'url'},
#                {'headerName':'Organisation','field':'organiser'},
#                {'headerName':'Ort','field':'location'},
#                {'headerName':'regelmäßig','field':'periodic', 'editable': True, 'cellDataType': 'boolean'},
#               ]

# grid_options = {
#     'columnDefs' : column_defs,
#     'enableSorting': True,
#     'enableFilter': True,
#     'enableColResize': True,
#     'enableRangeSelection': True,
# }

# g = Grid(grid_data=events,
#          grid_options=grid_options,
#          theme='ag-theme-balham',
#          quick_filter=True,
#          show_toggle_edit=True,
#          # export_mode="buttons",
#          export_csv=True,
#          export_excel=True,
#          # show_toggle_delete=True,
#          sync_on_edit=True,
#          columns_fit='auto',
#          index=False,
#          keep_multiindex=False,)
# g

In [ ]:
# g.grid_data_out.get('grid')